In [0]:
# create autoloader _input folder in volume
dbutils.fs.mkdirs("/Volumes/dev/bronze/landing/autoloader_input/2010/12/01")
dbutils.fs.mkdirs("/Volumes/dev/bronze/landing/autoloader_input/2010/12/02")
dbutils.fs.mkdirs("/Volumes/dev/bronze/landing/autoloader_input/2010/12/03")
dbutils.fs.mkdirs("/Volumes/dev/bronze/landing/autoloader_input/2010/12/04")
dbutils.fs.mkdirs("/Volumes/dev/bronze/landing/autoloader_input/2010/12/05")
dbutils.fs.mkdirs("/Volumes/dev/bronze/landing/autoloader_input/2010/12/06")
dbutils.fs.mkdirs("/Volumes/dev/bronze/landing/autoloader_input/2010/12/07")

In [0]:
# Create checkpoint location in Volume
dbutils.fs.mkdirs("/Volumes/dev/bronze/landing/checkpoint/autoloader")

In [0]:
# Copy files to nested location

dbutils.fs.cp("/databricks-datasets/definitive-guide/data/retail-data/by-day/2010-12-01.csv", "/Volumes/dev/bronze/landing/autoloader_input/2010/12/01")
dbutils.fs.cp("/databricks-datasets/definitive-guide/data/retail-data/by-day/2010-12-02.csv", "/Volumes/dev/bronze/landing/autoloader_input/2010/12/02")
dbutils.fs.cp("/databricks-datasets/definitive-guide/data/retail-data/by-day/2010-12-03.csv", "/Volumes/dev/bronze/landing/autoloader_input/2010/12/03")

In [0]:
df = (
    spark
    .readStream
    .format("cloudFiles")
    .option("cloudFiles.format", "csv")
    .option("pathGlobFilter", "*.csv")
    .option("header", "true")
    .option("cloudFiles.schemaEvolutionMode", "none")
    .option("cloudFiles.schemaLocation", "/Volumes/dev/bronze/landing/checkpoint/autoloader/3/")
    .option("cloudFiles.schemaHints", "Quantity int, UnitPrice double")
    .load("/Volumes/dev/bronze/landing/autoloader_input/*/")
)

In [0]:
# write data to delta table- dev.bronze.invoice_al_1
from pyspark.sql.functions import col

(
    df.withColumn("__file", col("_metadata.file_name"))
    .writeStream
    .option("checkpointLocation", "/Volumes/dev/bronze/landing/checkpoint/autoloader/3/")
    .option("mergeSchema", "true")
    .outputMode("append")
    .trigger(availableNow=True)
    .toTable("dev.bronze.invoice_al_3")
)

In [0]:
%sql
select * from dev.bronze.invoice_al_3;

In [0]:
%sql
select __file, count(1)
from dev.bronze.invoice_al_3
group by __file
;


In [0]:
%sql
select *
from dev.bronze.invoice_al_3
where __file = '2010-12-06.csv'

In [0]:
dbutils.fs.cp("/databricks-datasets/definitive-guide/data/retail-data/by-day/2010-12-05.csv", "/Volumes/dev/bronze/landing/autoloader_input/2010/12/05")

In [0]:
%sql
select *
from dev.bronze.invoice_al_1 where InvoiceDate like "2010-12-03 09:31%"
limit 4

In [0]:
sixth_file_df = spark.sql("""
                          select * from dev.bronze.invoice_al_1 where InvoiceDate like "2010-12-03 09:31%" limit 4
                          """)

sixth_file_df.show()

In [0]:
from pyspark.sql.functions import lit
display(sixth_file_df.withColumn("State", lit("UK")).select("InvoiceNo", "StockCode", "Description", "Quantity", "InvoiceDate", "UnitPrice", "CustomerID", "Country", "State"))